最近总是接触各种
Jupyter…（JupyterNotebook/JupyterHub），发现居然还有一个可以完全跑在浏览器上的
[Jupyterlite](https://github.com/jupyterlite/jupyterlite)，那是不是可以直接放一个到这儿呢？开干

## 大致的概念

原来的 JupyterNotebook 是传统的 CS 模式，各种语言（e.g. python）的
kernel 是跑在服务端的

但现在 python（和一些常用的库）也有了 WebAssembly 的
port（e.g. [pyodide](https://pyodide.org/en/stable/)）， 这样 kernel
也可以在浏览器上实现了，于是可以创建一个全静态的 site，UI 和 kernel
都在浏览器中运行， 中间用 mock-socket 假装通讯，这样就可以把 notebook
那一套 CS 模式移植过来

## 大致的流程

1.  安装 jupyterlite
2.  用装好的 jupyterlite 生成一个 site

## 具体的流程

### 安装 jupyterlite

其实就是 `pip install --pre jupyterlite`（现在还处于
[pre-release](https://github.com/jupyterlite/jupyterlite/releases/tag/v0.1.0b12)
的状态）

### 生成 site

运行装好的 jupyterlite：`jupyter lite build`，该命令会在
[LiteDir](https://jupyterlite.readthedocs.io/en/latest/reference/cli.html#the-lite-dir)（默认当前目录）
读取配置（默认为 `jupyter_lite_config.json`[1]），复制内容 （例如
LiteDir 下如果有 `files` 目录则会直接复制到输出目录[2]）等，
然后输出到一个目录（默认为 `_output`），该目录下就是 site 了

### 例子

以下是一个 build script 和配置：它会首先创建 python virutal env 然后安装
jupyterlite 最后 build 一个 site 到目录 `jl` 中， 这里根据 [Create a
JupyterLite archive that can be used
offline](https://jupyterlite.readthedocs.io/en/latest/howto/configure/advanced/offline.html)
下载完整的 pyodide 以及 mathjax（不需要用 CDN），另外也添加了预下载的
`toml-0.10.2` wheel（不需要从 pypi 下载）， 这样创建出来的 site
即使完全不联网也是能运作的，只是体积比较大

-   `build.sh`

    ``` bash
    #!/bin/bash

    if [ ! -d venv ]; then
      echo "python venv not found, create a new one"
      python3 -m venv venv
    else
      echo "python venv found"
    fi

    echo "activating venv"
    source venv/bin/activate

    echo "pip install jupyterlite[mathjax]"
    pip install --pre jupyterlite[mathjax]

    echo "pip install jupyterlite[piplite]"
    pip install --pre jupyterlite[piplite]

    PYODIDE_VER=0.21.1

    if [ ! -f pyodide.tar.bz2 ]; then
      echo "pyodide not found, try to download... (or you can manally download and name it pyodide.tar.bz2)"
      wget --verbose -O pyodide.tar.bz2 \
        https://github.com/pyodide/pyodide/releases/download/${PYODIDE_VER}/pyodide-build-${PYODIDE_VER}.tar.bz2
    else
      echo "pyodide found"
    fi

    echo "jupyter lite build... (using jupyter_lite_config.json)"
    jupyter lite build
    ```

-   `jupyter_lite_config.json`

    ``` json
    {
      "LiteBuildConfig": {
        "output_dir": "jl",
        "pyodide_url": "pyodide.tar.bz2",
        "piplite_urls": [
          "https://files.pythonhosted.org/packages/44/6f/7120676b6d73228c96e17f1f794d8ab046fc910d781c8d151120c3f1569e/toml-0.10.2-py2.py3-none-any.whl"
        ]
      }
    }
    ```

### 部署

部署方式有多种，例如可以在 github 上新建一个名为 `jupyterlite`
的项目，把刚才生成的 site push 上去，并且设置 github page，
这样它就可以在 `<username>.github.io/jupyterlite` 这个路径上访问得到了

### 内容

如上所述，可将内容（`*.ipynb`）放置到 LiteDir 的 files 里，然后再次跑
build script 即可更新 site 中的内容

### 不同之处

原 CS 模式下，packages 是在服务器上安装好，可以直接 `import`，而在
jupyterlite 中，似乎除 pyodide 自带的那些包外， 需要首先
`piplite.install` 才能 `import`，例如：

[1] 在这里可以看到一个官方的配置例子
<https://github.com/jupyterlite/jupyterlite/blob/main/examples/jupyter_lite_config.json>

另外运行 `jupyter lite -h` 也可以看到具体说明

``` shell
$ jupyter lite -h
...
--output-dir=<CPath>
    Where to build the JupyterLite site. env: JUPYTERLITE_OUTPUT_DIR
    Default: traitlets.Undefined
    Equivalent to: [--LiteBuildConfig.output_dir]
...
--piplite-wheels=<typedtuple-item-1>...
    Local paths or URLs of piplite-compatible wheels to copy and index
    Default: ()
    Equivalent to: [--LiteBuildConfig.piplite_urls]
...
--pyodide=<Unicode>
    Local path or URL of a pyodide distribution tarball
    Default: ''
    Equivalent to: [--LiteBuildConfig.pyodide_url]
```

[2] <https://jupyterlite.readthedocs.io/en/latest/howto/content/files.html>

In [ ]:
import toml
toml

这是会失败的

In [ ]:
import piplite
piplite.install('toml')
import toml
toml

这样才行

注：上面将 whl url 写到 `piplite_urls` 中只是将 wheel 提前下载到 site
中，并告诉 jupyterlite 如果要 piplite install 这个包则不要去 pypi
找而是在本地找，但要使用它之前还是需要 `piplite.install` 的